# AutoEncoders

## Downloading the dataset

In [1]:
###ML-100K
'''
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls
'''
###ML-1M"""
'''
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls
'''

'\n!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"\n!unzip ml-1m.zip\n!ls\n'

## Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset

In [3]:
# We won't be using this dataset.
movies = pd.read_csv("ml-1m/movies.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
users = pd.read_csv("ml-1m/users.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
ratings = pd.read_csv("ml-1m/ratings.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")

## Preparing the training set and the test set

In [4]:
training_set = pd.read_csv("ml-100k/u1.base", delimiter = '\t')
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("ml-100k/u1.test", delimiter = '\t')
test_set = np.array(test_set, dtype = "int")

## Getting the number of users and movies

In [5]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns

In [6]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings # id_movies start from 1, array index start from 0
        new_data.append(list(ratings)) # convert np array to list
    return new_data

In [7]:
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network

In [9]:
class SAE(nn.Module): # stacked autoencoder, inherited from nn.Module
    def __init__(self, ): # ',' for parameters from inheritance
        super(SAE, self).__init__() # get all classes and modules from inheritance
        self.fc1 = nn.Linear(nb_movies, 20) # First full connection layer (fcl - encoder)
        #                        |       |___ Number of nodes in first hidden layer (experiment with different numbers)
        #                        |___________ Number of nodes in first fully connected layer
        self.fc2 = nn.Linear(20, 10) # Second full connection layer (fc2 - encoder)
        #                    |    |___ Number of nodes in second hidden layer (experiment with different numbers)
        #                    |________ Number of nodes in first hidden layer
        self.fc3 = nn.Linear(10, 20) # Third full connection layer (fc3 - decoder)
        #                    |   |____ Number of nodes in second hidden layer (experiment with different numbers)
        #                    |________ Number of nodes in third hidden layer
        self.fc4 = nn.Linear(20, nb_movies) # Fourth full connection layer (fc4 - decoder)
        #                    |   |___________ Number of nodes in fourth hidden layer (experiment with different numbers)
        #                    |_______________ Number of nodes in third hidden layer
        self.activation = nn.Sigmoid() # sigmoid activation functions(try sigmoid rectifier activation functions)

    def forward(self, x): # x : input vector
        x = self.activation(self.fc1(x)) # get activation of first hidden layer (from input vector), encoder
        x = self.activation(self.fc2(x)) # get activation of second hidden layer1, encoder
        x = self.activation(self.fc3(x)) # get activation of third hidden layer, decoder
        x = self.fc4(x) # get output vector, decoder
        return x

In [10]:
sae = SAE()
criterion = nn.MSELoss() # mean squared error loss function
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5) # root mean square prop
#                                   |        |              |___________ Reduce learning rate after every few epoch for regulate the convergence
#                                   |        |________ Learning rate
#                                   |_________________ Get all parameters form sae object

## Training the SAE

In [11]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0. # for normalize the train loss
    for id_user in range(nb_users):
        # pytorch need batch not vector
        #           |
        input = Variable(training_set[id_user]).unsqueeze(0)
        #                                            |______ put data into index zero of new dimension
        target = input.clone() # keep the copy of input
        if torch.sum(target.data > 0) > 0: # get users who at least rated one movie
            output = sae(input) # sae.forward(input)
            target.require_grad = False # do not compute gradiant with respect the target (for optimize)
            output[target == 0] = 0 # replace movies that not rated with zeros of output vector (for optimize)
            loss = criterion(output, target) # compute MSE loss
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # number of movies / number of movies have possitive (non zero) ratings
            #                                       |                        |____ For avoid zero division error
            #                                       |_____________________________ Get number of movies with possitive ratings
            loss.backward() # In which direction need to update the weights (increase or decrease)
            train_loss += np.sqrt(loss.data*mean_corrector) # update the loss in each epoch
            s += 1.
            optimizer.step() # update the weights with RMSprop
    print(f"epoch: {epoch} loss: {train_loss/s}")

epoch: 1 loss: 1.771559476852417
epoch: 2 loss: 1.0967310667037964
epoch: 3 loss: 1.0534603595733643
epoch: 4 loss: 1.0382895469665527
epoch: 5 loss: 1.0308328866958618
epoch: 6 loss: 1.0265097618103027
epoch: 7 loss: 1.0236423015594482
epoch: 8 loss: 1.02169930934906
epoch: 9 loss: 1.0207910537719727
epoch: 10 loss: 1.0193887948989868
epoch: 11 loss: 1.0190292596817017
epoch: 12 loss: 1.0183582305908203
epoch: 13 loss: 1.0178768634796143
epoch: 14 loss: 1.017232894897461
epoch: 15 loss: 1.0171600580215454
epoch: 16 loss: 1.016748309135437
epoch: 17 loss: 1.0167073011398315
epoch: 18 loss: 1.0162416696548462
epoch: 19 loss: 1.0162694454193115
epoch: 20 loss: 1.0159766674041748
epoch: 21 loss: 1.016108512878418
epoch: 22 loss: 1.0156826972961426
epoch: 23 loss: 1.0156831741333008
epoch: 24 loss: 1.0155222415924072
epoch: 25 loss: 1.015668511390686
epoch: 26 loss: 1.0153682231903076
epoch: 27 loss: 1.015243411064148
epoch: 28 loss: 1.014931321144104
epoch: 29 loss: 1.0131922960281372
epo

## Testing the SAE

In [12]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0) # model predict according to input training data by users and give ratings for movies which are not rated by the user base on rated movies from training set
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input) # predict base on training set data
    target.require_grad = False
    output[target == 0] = 0 # Compare only with rated movies on test set
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print(f"test loss: {test_loss/s}")

test loss: 0.9521654844284058
